In [1]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, InputLayer, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import GridSearchCV
#from keras.models import Sequential
#from keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow
import warnings
tensorflow.random.set_seed(42)
warnings.filterwarnings("ignore", category=FutureWarning)

Create the model  as per the configuration provided in https://doi.org/10.1021/acs.jcim.8b00685

In [2]:
def create_model(learn_rate):
    model = Sequential()
    # this model has two hidden layers and two dropout layers
	# relu activation function is used at both hidden layers
    hidden_layer = Dense(512, activation='relu')
    model.add(hidden_layer)
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    hidden_layer_2 = Dense(256, activation='relu')
    model.add(hidden_layer_2)
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    hidden_layer_3 = Dense(128, activation='relu')
    model.add(hidden_layer_3)
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    hidden_layer_4 = Dense(64, activation='relu')
    model.add(hidden_layer_4)
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    hidden_layer_5 = Dense(32, activation='relu')
    model.add(hidden_layer_5)
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    hidden_layer_6 = Dense(32, activation='relu')
    model.add(hidden_layer_6)
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))
    # using an adam optimizer with a learning rate of 0.005
	# using mean-squared error as our loss function and mean average error as our metri
    opt = Adam(learning_rate=learn_rate)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

Upload sub-train dataset and get dependent and independent parameters

In [3]:
df=pd.read_csv('PS3M_best_train.csv')
y=df.iloc[:,1:2]
X=df.iloc[:,2:]

Convert the dependent parameter y from binary to categorical array

In [4]:
le=LabelEncoder()
y=le.fit_transform(y.astype(int))
y=to_categorical(y, num_classes=None, dtype = 'int64')

C:\Users\amitc\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Perform the grid search with the created DNN model

In [5]:
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10,verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.5]
batch_size = [50,100,200]
epochs = [50,100,200]
param_grid = dict(learn_rate=learn_rate, batch_size=batch_size, epochs=epochs)
#param_grid = dict(batch_size=batch_size, learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=5)
grid_result = grid.fit(X.values, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

C:\Users\amitc\AppData\Local\Temp/ipykernel_9048/2252741717.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10,verbose=0)


Best: 0.878505 using {'batch_size': 50, 'epochs': 200, 'learn_rate': 0.01}
0.866801 (0.017138) with: {'batch_size': 50, 'epochs': 50, 'learn_rate': 0.001}
0.870702 (0.012334) with: {'batch_size': 50, 'epochs': 50, 'learn_rate': 0.01}
0.800629 (0.052087) with: {'batch_size': 50, 'epochs': 50, 'learn_rate': 0.1}
0.659649 (0.021533) with: {'batch_size': 50, 'epochs': 50, 'learn_rate': 0.5}
0.878499 (0.010906) with: {'batch_size': 50, 'epochs': 100, 'learn_rate': 0.001}
0.876158 (0.014317) with: {'batch_size': 50, 'epochs': 100, 'learn_rate': 0.01}
0.805174 (0.078407) with: {'batch_size': 50, 'epochs': 100, 'learn_rate': 0.1}
0.659649 (0.021533) with: {'batch_size': 50, 'epochs': 100, 'learn_rate': 0.5}
0.875383 (0.015810) with: {'batch_size': 50, 'epochs': 200, 'learn_rate': 0.001}
0.878505 (0.011900) with: {'batch_size': 50, 'epochs': 200, 'learn_rate': 0.01}
0.659649 (0.021533) with: {'batch_size': 50, 'epochs': 200, 'learn_rate': 0.1}
0.659649 (0.021533) with: {'batch_size': 50, 'epoch

Fit the model with optimized parameters

In [6]:
model = KerasClassifier(build_fn = lambda: create_model(learn_rate = 0.01), epochs = 200, batch_size = 50, verbose = 0)

C:\Users\amitc\AppData\Local\Temp/ipykernel_9048/208264848.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = lambda: create_model(learn_rate = 0.01), epochs = 200, batch_size = 50, verbose = 0)


In [7]:
model.fit(X.values, y, verbose=1)

Epoch 1/200
26/26 [==============================] - 1s 9ms/step - loss: 0.6795 - accuracy: 0.6519
Epoch 2/200
26/26 [==============================] - 0s 7ms/step - loss: 0.5411 - accuracy: 0.7251
Epoch 3/200
26/26 [==============================] - 0s 6ms/step - loss: 0.5007 - accuracy: 0.7453
Epoch 4/200
26/26 [==============================] - 0s 6ms/step - loss: 0.4399 - accuracy: 0.7780
Epoch 5/200
26/26 [==============================] - 0s 6ms/step - loss: 0.4621 - accuracy: 0.7960
Epoch 6/200
26/26 [==============================] - 0s 6ms/step - loss: 0.4287 - accuracy: 0.8240
Epoch 7/200
26/26 [==============================] - 0s 6ms/step - loss: 0.4306 - accuracy: 0.8209
Epoch 8/200
26/26 [==============================] - 0s 7ms/step - loss: 0.4200 - accuracy: 0.8302
Epoch 9/200
26/26 [==============================] - 0s 6ms/step - loss: 0.4117 - accuracy: 0.8318
Epoch 10/200
26/26 [==============================] - 0s 6ms/step - loss: 0.4216 - accuracy: 0.8271
Epoch 11/

26/26 [==============================] - 0s 6ms/step - loss: 0.3281 - accuracy: 0.8629
Epoch 84/200
26/26 [==============================] - 0s 6ms/step - loss: 0.3301 - accuracy: 0.8777
Epoch 85/200
26/26 [==============================] - 0s 5ms/step - loss: 0.3581 - accuracy: 0.8528
Epoch 86/200
26/26 [==============================] - 0s 6ms/step - loss: 0.3526 - accuracy: 0.8660
Epoch 87/200
26/26 [==============================] - 0s 5ms/step - loss: 0.3506 - accuracy: 0.8583
Epoch 88/200
26/26 [==============================] - 0s 5ms/step - loss: 0.3356 - accuracy: 0.8738
Epoch 89/200
26/26 [==============================] - 0s 6ms/step - loss: 0.3391 - accuracy: 0.8676
Epoch 90/200
26/26 [==============================] - 0s 5ms/step - loss: 0.3436 - accuracy: 0.8676
Epoch 91/200
26/26 [==============================] - 0s 6ms/step - loss: 0.3164 - accuracy: 0.8801
Epoch 92/200
26/26 [==============================] - 0s 6ms/step - loss: 0.3424 - accuracy: 0.8692
Epoch 93/200


In [18]:
from sklearn import metrics
from sklearn.metrics import accuracy_score,f1_score,matthews_corrcoef,recall_score,roc_auc_score,roc_curve,confusion_matrix
import numpy as np
import math
from sklearn.model_selection import KFold  #For K-fold cross validation\n
import random
import copy
import math
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from numpy import ndarray
from sklearn.metrics import accuracy_score, matthews_corrcoef

class cross_validation:
      def __init__(self,X_data,y_data,model,cv):
            self.X_data=X_data
            self.y_data=y_data
            self.cv=cv
            self.model=model
            #self.epochs=epochs
            #self.batch_size=batch_size
                
      def fit(self):
         kf = KFold(n_splits=self.cv, random_state=None, shuffle=False)
         C,tp,tn,fp,fn,pr,rc,mcc = [],[],[],[],[],[],[],[]
         for train, test in kf.split(self.X_data):
            train_predictors = (self.X_data.iloc[train,:])
            train_target = pd.DataFrame(self.y_data).iloc[train]
            self.model.fit(train_predictors,np.array(train_target)) ##model fitting
            tspred = self.model.predict(self.X_data.iloc[test,:])
            #tspred1=np.argmax(tspred, axis=1)
            y_data_processed=np.argmax(np.array(pd.DataFrame(self.y_data).iloc[test]),axis=1)
            cm1=confusion_matrix(y_data_processed, tspred)
            tp.append(cm1[1,1])
            tn.append(cm1[0,0])
            fp.append(cm1[0,1])
            fn.append(cm1[1,0])
      
                  
         TPs=np.sum(tp)
         TNs=np.sum(tn)
         FPs=np.sum(fp)
         FNs=np.sum(fn)
         Sn=(TPs/(TPs+FNs))*100
         Sp=(TNs/(TNs+FPs))*100
         PRs=(TPs/(TPs+FPs))*100
         F1s=(2*TPs / (2*TPs + FPs + FNs))*100
         Acc=((TPs+TNs)/(TPs+TNs+FPs+FNs))*100
         print('True positive: '+str(TPs))
         print('True negative: '+str(TNs))
         print('False postive: '+str(FPs))
         print('False negative: '+str(FNs))
         print('Sensitivity: '+str(Sn))
         print('Specificity: '+str(Sp))
         print('Accuracy: '+str(Acc))
         print('Precision: '+str(PRs))
         print('F1 score: '+str(F1s))
         return TPs,TNs,FPs,FNs,Sn,Sp,Acc,F1s

In [19]:
cvv=cross_validation(pd.DataFrame(X),y,model,5)
cvv.fit()

True positive: 804
True negative: 318
False postive: 119
False negative: 43
Sensitivity: 94.9232585596222
Specificity: 72.76887871853546
Accuracy: 87.38317757009347
Precision: 87.10725893824485
F1 score: 90.84745762711864


(804,
 318,
 119,
 43,
 94.9232585596222,
 72.76887871853546,
 87.38317757009347,
 90.84745762711864)

In [20]:
ts=pd.read_csv('PS3M_best_test.csv')
yts=ts.iloc[:,1:2]
Xts=ts.iloc[:,2:].values
yts=le.fit_transform(yts.astype(int))
yts=to_categorical(yts, num_classes=None, dtype = 'int64') 
ytspr=model.predict(Xts)
ytspr=pd.DataFrame(ytspr, columns=['Pred']) 
yts1=np.argmax(yts, axis=1)
cm1=confusion_matrix(yts1,ytspr)
Sn = float(cm1[0,0])/(cm1[0,0]+cm1[0,1])
Sp=float(cm1[1,1])/(cm1[1,0]+cm1[1,1])
Sn=Sn*100
Sp=Sp*100
acc=accuracy_score(yts1,ytspr)*100
f1=f1_score(yts1,ytspr)*100
mcc=matthews_corrcoef(yts1,ytspr)
roc=roc_auc_score(yts1,ytspr)
print('True Positive: '+str(cm1[1,1]))
print('True Negative: '+str(cm1[0,0]))
print('False Positive '+str(cm1[0,1]))
print('False Negative '+str(cm1[1,0]))
print('Sensitivity: '+str(Sn))
print('Specificity: '+str(Sp))
print('Accuracy: '+str(acc))
print('f1_score: '+str(f1))
#print('Recall score: '+str(recall_score(self.y,ypred)))
print('MCC: '+str(mcc))
print('ROC_AUC: '+str(roc))

True Positive: 182
True Negative: 97
False Positive 34
False Negative 8
Sensitivity: 74.04580152671755
Specificity: 95.78947368421052
Accuracy: 86.91588785046729
f1_score: 89.65517241379311
MCC: 0.7315866505150264
ROC_AUC: 0.8491763760546404


C:\Users\amitc\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [21]:
vd=pd.read_csv('PS3M_best_validation.csv')
yvd=vd.iloc[:,1:2]
Xvd=vd.iloc[:,2:].values
yvd=le.fit_transform(yvd.astype(int))
yvd=to_categorical(yvd, num_classes=None, dtype = 'int64') 
yvdpr=model.predict(Xvd)
yvdpr=pd.DataFrame(yvdpr, columns=['Pred']) 
yvd1=np.argmax(yvd, axis=1)
cm1=confusion_matrix(yvd1,yvdpr)
Sn = float(cm1[0,0])/(cm1[0,0]+cm1[0,1])
Sp=float(cm1[1,1])/(cm1[1,0]+cm1[1,1])
Sn=Sn*100
Sp=Sp*100
acc=accuracy_score(yvd1,yvdpr)*100
f1=f1_score(yvd1,yvdpr)*100
mcc=matthews_corrcoef(yvd1,yvdpr)
roc=roc_auc_score(yvd1,yvdpr)
print('True Positive: '+str(cm1[1,1]))
print('True Negative: '+str(cm1[0,0]))
print('False Positive '+str(cm1[0,1]))
print('False Negative '+str(cm1[1,0]))
print('Sensitivity: '+str(Sn))
print('Specificity: '+str(Sp))
print('Accuracy: '+str(acc))
print('f1_score: '+str(f1))
#print('Recall score: '+str(recall_score(self.y,ypred)))
print('MCC: '+str(mcc))
print('ROC_AUC: '+str(roc))

True Positive: 426
True Negative: 175
False Positive 78
False Negative 17
Sensitivity: 69.1699604743083
Specificity: 96.16252821670429
Accuracy: 86.35057471264368
f1_score: 89.96832101372756
MCC: 0.7031135104815757
ROC_AUC: 0.8266624434550629


C:\Users\amitc\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
